# Mnist classification demo

In [2]:
import grpc 
import numpy as np
import hydro_serving_grpc as hs
from grpc import ssl_channel_credentials
import keras
from keras.datasets import mnist
from keras import backend as K
import os

/Users/nikitagurov/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/nikitagurov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Read data sample

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_rows, img_cols = 28, 28
num_classes = 10

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## Setup channel

In [8]:
channel = grpc.secure_channel("dev.k8s.hydrosphere.io", credentials=ssl_channel_credentials())
stub = hs.PredictionServiceStub(channel) 
model_spec = hs.ModelSpec(name="mnist2", signature_name="INFERRED_PIPELINE_SIGNATURE")

## Create tensors

In [19]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=1), hs.TensorShapeProto.Dim(size=28), hs.TensorShapeProto.Dim(size=28)])
tensor = hs.TensorProto(dtype=hs.DT_DOUBLE, tensor_shape=tensor_shape, double_val=np.array([x_test[0]]).flatten().astype(np.float32))

## Send request to model servable

In [20]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"input": tensor}) 
result = stub.Predict(request)
print('Output values:')
print(result)
print('Ground truth label:')
print(y_test[0])

Output values:
outputs {
  key: "classes"
  value {
    dtype: DT_INT64
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
    int64_val: 0
    int64_val: 0
    int64_val: 0
    int64_val: 0
    int64_val: 0
    int64_val: 0
    int64_val: 0
    int64_val: 0
    int64_val: 0
    int64_val: 0
  }
}
outputs {
  key: "probabilities"
  value {
    dtype: DT_DOUBLE
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 10
      }
    }
    double_val: 2.9983486456330866e-05
    double_val: 1.495610413293491e-09
    double_val: 4.615424040821381e-05
    double_val: 0.0024091629311442375
    double_val: 1.2768715578204137e-06
    double_val: 2.4745755581534468e-05
    double_val: 7.330490880974594e-09
    double_val: 0.996881365776062
    double_val: 2.5152998205157928e-05
    double_val: 0.0005822270177304745
  }
}

Ground truth label:
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
